***SAC** 算法也可以处理 **离散动作交互（车杆环境）** 的环境，但是要做一些相应的修改：*

导入相关库：

In [1]:
# 基本库
import numpy as np

from utils.replay_buffer import ReplayBuffer
from utils.training import train_off_policy_agent
from utils.smoothing import moving_average
# 神经网络
import torch
import torch.nn.functional as F

from torch.distributions import Normal
# Gymnasium 是一个用于开发和测试强化学习算法的工具库，为 OpenAI Gym 的更新版本（2021迁移开发）
import gymnasium as gym

### 定义策略网络和价值网络:
- **策略网络** 的输出修改为在离散动作空间上的 **softmax 分布**
- **价值网络** 直接接收状态和离散动作空间的分布作为输入

In [ ]:
class PolicyNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(PolicyNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return F.softmax(self.fc2(x), dim=1)


class QValueNet(torch.nn.Module):
    def __init__(self, state_dim, hidden_dim, action_dim):
        super(QValueNet, self).__init__()
        self.fc1 = torch.nn.Linear(state_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, action_dim)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        return self.fc2(x)